# - LABORATORIO SPARK, CORRESPONDIENTE A LA 2DA PARCIAL

Asignatura: Mineria de Datos

Docente   : Carlos Fernando Montoya Cubas

Estudiante: Vladimir Dante Casilla Percca

#Spark + Python = PySpark

Este notebook presenta los conceptos básicos de Spark a través de su interfaz de lenguaje Python. Como aplicación inicial usaremos el ejemplo clásico del contador de palabras. Con este ejemplo, puede comprender la lógica de programación funcional para las diversas tareas de exploración de datos distribuidos.

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 33 kB/s 
     |████████████████████████████████| 198 kB 66.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=f472c2b3f9ce88af831af3c56fdab9b749bcea77991306c7b67dbd144cc6436a
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


Parte 1: Creación y manipulación de RDD

En esta parte del notebook crearemos una base RDD a partir de una lista con el comando parallelize.

#(1a) Creando una base RDD

Podemos crear una base de datos RDD de diferentes tipos y fuente Python con el comando sc.parallelize (fuente, particiones), siendo fuente una variable que contiene los datos (ej .: una lista) y particiona el número de particiones trabajar en paralelo.

#Instalacion de pySpark en Google Colaboratory

In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [3]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print(type(palavrasRDD))

<class 'pyspark.rdd.RDD'>


(1b) Plural

Vamos a crear una función que transforme una palabra en plural agregando una letra 's' al final de la cadena. A continuación, usemos la función map() para aplicar la transformación a cada palabra del RDD.

In [4]:
# EXERCICIO
def Plural(palavra):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return f"{palavra}s"
print(Plural('gato'))

gatos


In [5]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
    
    Returns:
        str: A string with 's' added to it.



In [6]:
assert Plural('rato')=='ratos', 'resultado incorreto!'
print ('OK')

OK


(1c) Aplicar la función a RDD

Pluralice cada palabra de nuestro RDD usando [map()] (http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map)

In [7]:
# EXERCICIO
pluralRDD = palavrasRDD.map(Plural)
print (pluralRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [8]:
assert pluralRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print ('OK')

OK


Nota: use el comando collect() solo cuando esté seguro de que la lista cabe en la memoria. Para guardar los resultados en un archivo de texto o base de datos, usaremos otro comando.

(1d) Usando una función lambda

In [9]:
# EXERCICIO
pluralLambdaRDD = palavrasRDD.map(lambda x: f"{x}s")
print (pluralLambdaRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [10]:
assert pluralLambdaRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print ('OK')

OK


(1e) Tamaño de cada palabra

Ahora usa map() y una función lambda para devolver el número de caracteres en cada palabra. Utilice collect() para almacenar el resultado como listas en la variable de destino.

In [11]:
# EXERCICIO
pluralTamanho = (pluralRDD
                 .map(lambda x:len(x))
                 .collect()
                 )
print (pluralTamanho)

[5, 9, 5, 5, 5]


In [12]:
assert pluralTamanho==[5,9,5,5,5], 'valores incorretos'
print ("OK")

OK


(1f) RDD de pares y tuplas

Para contar la frecuencia de cada palabra de forma distribuida, primero debemos asignar un valor a cada palabra en el RDD. Esto generará una base de datos (clave, valor). De esta forma podemos agrupar la base mediante la clave, calculando la suma de los valores asignados. En nuestro caso, asignemos el valor 1 a cada palabra.

In [13]:
# EXERCICIO
palavraPar = palavrasRDD.map(lambda x: (x, 1))
print (palavraPar.collect())

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]


In [14]:
assert palavraPar.collect() == [('gato',1),('elefante',1),('rato',1),('rato',1),('gato',1)], 'valores incorretos!'
print ("OK")

OK


#Parte 2: Manipulación de Tuplas RDD

Manipulemos nuestro RDD para contar las palabras en el texto.
(2a) Función groupByKey ()

In [15]:
# EJERCICIO
palavrasGrupo = palavraPar.groupByKey()
for chave, valor in palavrasGrupo.collect():
    valores = list(valor)
    print(f'{chave}: {valores}')

elefante: [1]
rato: [1, 1]
gato: [1, 1]


In [16]:
assert sorted(palavrasGrupo.mapValues(lambda x: list(x)).collect()) == [('elefante', [1]), ('gato',[1, 1]), ('rato',[1, 1])], 'Valores incorretos!'
print ("OK")

OK


(2b) Cálculo de los recuentos

Después de groupByKey() nuestro RDD contiene elementos compuestos por la palabra, como clave, y un iterador que contiene todos los valores correspondientes a esa clave.

In [17]:
# EXERCICIO
contagemGroup = palavrasGrupo.map(lambda x:(x[0],sum(x[1])))
print (contagemGroup.collect())

[('elefante', 1), ('rato', 2), ('gato', 2)]


(2c) reduceByKey

In [18]:
assert sorted(contagemGroup.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


(2d) Comandos de agrupación

La forma más común de realizar esta tarea, a partir de nuestras RDD RDDwords, es encadenar el mapa y los comandos reduceByKey en una línea de comando.

In [19]:
# EXERCICIO
contagem = palavraPar.reduceByKey(lambda x,y:x+y)
print( contagem.collect())

[('elefante', 1), ('rato', 2), ('gato', 2)]


In [20]:
assert sorted(contagem.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


(2d) Comandos de agrupación

La forma más común de realizar esta tarea, a partir de nuestras RDD RDDwords, es encadenar el mapa y los comandos reduceByKey en una línea de comando.

In [21]:
# EXERCICIO
contagemFinal = (palavrasRDD
                 .map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
                 .collect())
print (contagemFinal)

[('elefante', 1), ('rato', 2), ('gato', 2)]


In [22]:
assert sorted(contagemFinal)==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


# Parte 3: Encontrar las palabras únicas y calcular el recuento promedio

(3a) Palabras únicas

Calcula el número de palabras únicas en RDD. Utilice la misma canalización que antes pero reemplazando .collect() con otro método de API RDD.

In [23]:
# EXERCICIO
palavrasUnicas = palavrasRDD.distinct().count()
print (palavrasUnicas)

3


In [24]:
assert palavrasUnicas==3, 'valor incorreto!'
print ("OK")

OK


(3b) Cálculo del recuento promedio de palabras

Encuentra la frecuencia promedio de palabras usando RDD count.

In [25]:
# EXERCICIO
# add é equivalente a lambda x,y: x+y
from operator import add
total = (contagem
         .map(lambda x: x[1])
         .reduce(add)) 
media = total / float(palavrasUnicas)
print (total)
print (round(media, 2))

5
1.67


In [26]:
assert round(media, 2)==1.67, 'valores incorretos!'
print ("OK")


OK


#Parte 4: Aplicar nuestro algoritmo a un archivo

(4a) Función contaPalavras

Para aplicar nuestro algoritmo genéricamente a múltiples RDD, primero creemos una función para aplicarla a cualquier fuente de datos. Esta función recibe la entrada de un RDD que contiene una lista de claves (palabras) y devuelve un RDD de tuplas con las claves y su recuento en ese RDD

In [27]:
# EXERCICIO
def contaPalavras(chavesRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        chavesRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return (chavesRDD
            .map(lambda x: (x, 1))
            .reduceByKey(lambda x, y: x + y))
print (contaPalavras(palavrasRDD).collect())

[('elefante', 1), ('rato', 2), ('gato', 2)]


In [28]:
assert sorted(contaPalavras(palavrasRDD).collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


(4b) Normalizando el texto

Cuando trabajamos con datos reales, normalmente necesitamos estandarizar los atributos de tal manera que se ignoren las diferencias sutiles debidas a errores de medición o diferencias de estandarización.

In [29]:
# EXERCICIO
import re
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()
print (removerPontuacao('Ola, quem esta ai??!'))
print (removerPontuacao(' Sem espaco e_sublinhado!'))

ola quem esta ai
sem espaco esublinhado


In [30]:
assert removerPontuacao(' O uso de virgulas, embora permitido, nao deve contar. ')=='o uso de virgulas embora permitido nao deve contar', 'string incorreta!'
print ("OK")

OK


#Parte 5: Similaridad entre Objetos

En esta parte del laboratorio aprenderemos a calcular la distancia entre atributos numéricos, categóricos y textuales.

(5a) Vectores en el espacio euclidiano

In [37]:
import numpy as np

# Vamos criar uma função pNorm que recebe como parâmetro p e retorna uma função que calcula a pNorma
def pNorm(p):
    """Generates a function to calculate the p-Norm between two points.

    Args:
        p (int): The integer p.

    Returns:
        Dist: A function that calculates the p-Norm.
    """

    def Dist(x,y):
        return np.power(np.power(np.abs(x-y),p).sum(),1/float(p))
    return Dist

In [38]:
# Vamos criar uma RDD com valores numéricos
np.random.seed(42)
numPointsRDD = sc.parallelize(enumerate(np.random.random(size=(10,100))))

In [39]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartPointsRDD = numPointsRDD.cartesian(numPointsRDD)

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartPointsParesRDD = cartPointsRDD.map(lambda x: ((x[0][0],x[1][0]), (x[0][1],x[1][1])))


# Aplique um mapa para calcular a Distância Euclidiana entre os pares
Euclid = pNorm(2)
distRDD = cartPointsParesRDD.map(lambda x: (x[0], Euclid(x[1][0],x[1][1])))

# Encontre a distância máxima, mínima e média, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o cálculo da min, max, mean
statRDD = distRDD.map(lambda x: x[1])

minv, maxv, meanv = statRDD.min(), statRDD.max(), statRDD.mean()
print (minv, maxv, meanv)

0.0 4.709048183663605 3.7511916889753705


In [40]:
assert (minv.round(2), maxv.round(2), meanv.round(2))==(0.0, 4.71, 3.75), 'Valores incorretos'
print ("OK")

OK


(5b) Valores Categóricos

Cuando nuestros objetos están representados por atributos categóricos, no tienen similitud espacial. Para calcular la similitud entre ellos podemos primero transformar nuestro vector de atributos en un vector binario indicando, para cada valor posible de cada atributo, si tiene ese atributo o no.

In [41]:
# Vamos criar uma função para calcular a distância de Hamming
def Hamming(x,y):
    """Calculates the Hamming distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        H (int): The Hamming distance between x and y.
    """
    return (x!=y).sum()

# Vamos criar uma função para calcular a distância de Jaccard
def Jaccard(x,y):
    """Calculates the Jaccard distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        J (int): The Jaccard distance between x and y.
    """
    return (x==y).sum()/float( np.maximum(x,y).sum() )

In [42]:
# Vamos criar uma RDD com valores categóricos
catPointsRDD = sc.parallelize(enumerate([['alto', 'caro', 'azul'],
                             ['medio', 'caro', 'verde'],
                             ['alto', 'barato', 'azul'],
                             ['medio', 'caro', 'vermelho'],
                             ['baixo', 'barato', 'verde'],
                            ]))

In [43]:
# EXERCICIO
# Crie um RDD de chaves únicas utilizando flatMap
chavesRDD = (catPointsRDD
             .flatMap(lambda x: [((x[0],xi),1) for xi in x[1]])
             .reduceByKey(lambda x,y: x)
             .map(lambda x: x[0])
             )

chaves = dict((v,k) for k,v in chavesRDD.collect())
nchaves = len(chaves)
print (chaves, nchaves)

{'alto': 2, 'caro': 0, 'baixo': 4, 'verde': 1, 'azul': 2, 'medio': 3, 'barato': 4, 'vermelho': 3} 8


In [44]:
assert chaves=={'alto': 2, 'caro': 0, 'baixo': 4, 'verde': 1, 'azul': 2, 'medio': 3, 'barato': 4, 'vermelho': 3}, 'valores incorretos!'
print ("OK")

assert nchaves==8, 'número de chaves incorreta'
print ("OK")

OK
OK


In [45]:
def CreateNP(atributos,chaves):  
    """Binarize the categorical vector using a dictionary of keys.

    Args:
        atributos (list): List of attributes of a given object.
        chaves (dict): dictionary with the relation attribute -> index

    Returns:
        array (np.array): Binary array of attributes.
    """
    
    array = np.zeros(len(chaves))
    for atr in atributos:
        array[ chaves[atr] ] = 1
    return array

# Converte o RDD para o formato binário, utilizando o dict chaves
binRDD = catPointsRDD.map(lambda rec: (rec[0],CreateNP(rec[1], chaves)))
binRDD.collect()

[(0, array([1., 0., 1., 0., 0., 0., 0., 0.])),
 (1, array([1., 1., 0., 1., 0., 0., 0., 0.])),
 (2, array([0., 0., 1., 0., 1., 0., 0., 0.])),
 (3, array([1., 0., 0., 1., 0., 0., 0., 0.])),
 (4, array([0., 1., 0., 0., 1., 0., 0., 0.]))]

In [46]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartBinRDD = binRDD.cartesian(binRDD)

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartBinParesRDD = cartBinRDD.map(lambda x: ((x[0][0],x[1][0]), (x[0][1],x[1][1])))


# Aplique um mapa para calcular a Distância de Hamming e Jaccard entre os pares
hamRDD = cartBinParesRDD.map(lambda x: (x[0], Hamming(x[1][0],x[1][1])))
jacRDD = cartBinParesRDD.map(lambda x: (x[0], Jaccard(x[1][0],x[1][1])))

# Encontre a distância máxima, mínima e média, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o cálculo da min, max, mean
statHRDD = hamRDD.map(lambda x: x[1])
statJRDD = jacRDD.map(lambda x: x[1])

Hmin, Hmax, Hmean = statHRDD.min(), statHRDD.max(), statHRDD.mean()
Jmin, Jmax, Jmean = statJRDD.min(), statJRDD.max(), statJRDD.mean()

print ("\t\tMin\tMax\tMean")
print ("Hamming:\t{:.2f}\t{:.2f}\t{:.2f}".format(Hmin, Hmax, Hmean ))
print ("Jaccard:\t{:.2f}\t{:.2f}\t{:.2f}".format( Jmin, Jmax, Jmean ))

		Min	Max	Mean
Hamming:	0.00	5.00	2.40
Jaccard:	0.60	4.00	1.90


In [47]:
assert (Hmin.round(2), Hmax.round(2), Hmean.round(2)) == (0.00,5.00,2.40), 'valores incorretos'
print ("OK")
assert (Jmin.round(2), Jmax.round(2), Jmean.round(2)) == (0.60,4.00,1.90), 'valores incorretos'
print ("OK")

OK
OK
